In [1]:
from IPython.display import display
import ipywidgets as widgets
from simpleac import SimPleAC
from monte_carlo import monte_carlo_results
#from datetime import datetime as dt
import time
import plotly.graph_objects as go

button = widgets.Button(description="Run Simulation")

#--CONDITION SPECIFIC--START
wing_length = widgets.FloatSlider(description = "Wing Length (m)", 
                                  value = 16)
wing_area = widgets.FloatSlider(description = "Wing Area (m^2)", 
                                value = 23)
fuel_volume_available = widgets.FloatSlider(description = "Fuel Volume Available (m^3)", 
                                            value = 0.6)
flight_speed = widgets.FloatSlider(description = "Flight Speed (m/s)", 
                                   value = 50)
levers = [wing_length, wing_area, fuel_volume_available, flight_speed]
#--CONDITION SPECIFIC--END

levers_text = []
for lever in levers:
    lever.style = {'description_width': '200px'}
    lever.layout.width = '800px'
    
    lever.continuous_update = True
    lever.readout = False
    lever_text = widgets.FloatText(
        disabled=False,
        continuous_update=True,
        margin = '0 80px 50px 0'
    )
    widgets.link((lever, 'value'), (lever_text, 'value'))
    levers_text.append(lever)
    levers_text.append(lever_text)

progress = widgets.FloatProgress(value=0.0,
                                 min=0.0,
                                 max=1.0, 
                                 description='Loading:',
                                 bar_style='info',
                                 orientation='horizontal')
progress.layout.visibility = 'hidden'

out = widgets.Output(layout={'width': '90%',
                             'height': '150px',
                             'border': '1px solid black'})
ifeas = widgets.Output(layout={'width': '30%',
                               'height': '90%',
                               'border': '1px solid black'})
with ifeas:
    print("Infeasible Conditions")

fig = go.FigureWidget()
fig.add_scatter();
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    yaxis=go.layout.YAxis(
        title_text="Failure Rate",
        range=[0,100]
    ),
    xaxis=go.layout.XAxis(
        title_text="Fuel Consumed (lbs)",
        range=[1000,1200]
    )
);
fig.data[0].mode = 'lines+markers';

diagram = go.FigureWidget();
diagram.add_scatter();
diagram.update_layout(
    autosize=False,
    width=800,
    height=800,
    yaxis=go.layout.YAxis(
        range=[-12,8]
    ),
    xaxis=go.layout.XAxis(
        range=[-10,10]
    )
);
diagram.data[0].mode = 'lines';

item_layout = widgets.Layout(margin='0 0 50px 0',
                             justify_content='space-around',
                             justify_items='center',
                             align_content='space-evenly',
                             align_items='center')


In [ ]:
def draw_diagram(wing_length=16, wing_area=23, fuel_volume_available=0.6):
    tail_width = 2
    alpha_fuel = 1.3
    alpha_wing = .5
    x = [
        (alpha_fuel*fuel_volume_available+tail_width), #0
        (alpha_fuel*fuel_volume_available+tail_width), #1
        (alpha_fuel*fuel_volume_available), #2
        (alpha_fuel*fuel_volume_available), #3
        (alpha_wing*wing_length), #4
        (alpha_wing*wing_length), #5
        (alpha_fuel*fuel_volume_available), #6
        (alpha_fuel*fuel_volume_available), #7
        0, #8
    ]
    
    x = x + [-1*i for i in x[7::-1]] + [(alpha_fuel*fuel_volume_available+tail_width)]
    wing_place = .7
    wing_taper = .6
    alpha_length = 1
    alpha_area = 2
    tail_length = 1
    tail_taper = .5
    nose_length = .2
    y = [
        -(wing_place*alpha_length*wing_length), #0
        -(wing_place*alpha_length*wing_length-tail_length), #1
        -(wing_place*alpha_length*wing_length-tail_length-tail_taper), #2
        -((1-wing_taper)*alpha_area*wing_area/wing_length), #3
        -((1-wing_taper)*alpha_area*wing_area/wing_length), #4
        0, #5
        (wing_taper*alpha_area*wing_area/wing_length), #6
        ((1-wing_place)*alpha_length*wing_length-nose_length), #7
        ((1-wing_place)*alpha_length*wing_length), #8
    ]
    y = y + y[7::-1] + [-(wing_place*alpha_length*wing_length)]
    return x, y

diagram.data[0].x, diagram.data[0].y = draw_diagram()

In [3]:
x = []
y = []
conds = []
iconds = []
times = []
start_time = time.time()#dt.now()

def on_button_clicked(b):
    m = SimPleAC()
    #--CONDITION SPECIFIC--START
    m.substitutions.update({
      "S": wing_area.value,
      "A": wing_length.value**2/float(wing_area.value),
      "V_{f_{avail}}": fuel_volume_available.value,
      "V": flight_speed.value
    })
    #--CONDITION SPECIFIC--END
    cond = str([lever.value for lever in levers])
    out.clear_output()
    if cond in iconds:
        with out:
            print(cond + " already tested")
            print("Infeasible conditions")
    elif cond in conds:
        with out:
            print(cond + " already tested")
            i = conds.index(cond)
            print("Fuel consumption: %i lbs" % x[i])
            print("    Failure rate: % 2.1f%% " % y[i])
    else:
        with out:
            print(cond)
        progress.value = 0
        progress.layout.visibility = None
        sol = m.localsolve(verbosity = 0)
        sol_wing_area = sol("S").magnitude
        sol_wing_length = sol("A").magnitude
        sol_fuel = sol("V_{f_{avail}}").magnitude
        diagram.data[0].x, diagram.data[0].y = draw_diagram(sol_wing_length, 
                                                            sol_wing_area, 
                                                            sol_fuel)
        performance, failure = monte_carlo_results(m, progress, out, sol=sol)
        if performance:
            x.append(performance)
            y.append(failure)
            conds.append(cond)
            times.append(time.time()-start_time)
            fig.data[0].x = x
            fig.data[0].y = y
            fig.data[0].hovertext = conds
            fig.data[0].marker=dict(
                size=10,
                color=times, #set color equal to a variable
                colorscale='Magma', # one of plotly colorscales
                showscale=False
            )
        else:
            with out:
                print("Infeasible Conditions")
            with ifeas:
                print(cond)
            iconds.append(cond)
    progress.layout.visibility = 'hidden'

button.on_click(on_button_clicked)

In [4]:
controls = widgets.VBox(levers_text + [button, fig, progress, out])

In [5]:
widgets.HBox([controls, diagram, ifeas], layout=item_layout)